In [1]:
# import packages
import pandas as pd, numpy as np
import json
# import files
import data as dt

1. Replicar 100 bps? -> **10 Basis points: dejar como parametro del backtest**
2. Determinar origin vs destination exchange -> **Kraken es origin, Bitfinex es destination**
3. Como replicar los niveles del origin en el destination (volumen)
4. Validar el método para determinar maker/taker fee: iterativo vs estático
5. Orden de llenado por tiempo de entrada.
6. Evolución de Orderbooks tipo bootstrap.

In [4]:
# load data
ob_krak, ob_bit = dt.read_jsonOB(file_name = 'orderbooks_05jul21.json')


In [15]:
# return JSON object as dictionary
orderbooks_data = ob_data
ob_bit = orderbooks_data['bitfinex']  # select specific exchange

# drop None keys
ob_bit = {i_key: i_value for i_key, i_value in ob_bit.items() if i_value is not None}

# create Dataframe and rearange columns
ob_bit = {i_ob: pd.DataFrame(ob_bit[i_ob])[['bid_size','bid','ask','ask_size']]
           if ob_bit[i_ob] is not None else None for i_ob in list(ob_bit.keys())}


ob_krak = orderbooks_data['kraken']  # select specific exchange

# drop None keys
ob_krak = {i_key: i_value for i_key, i_value in ob_krak.items() if i_value is not None}

# create Dataframe and rearange columns
ob_krak = {i_ob: pd.DataFrame(ob_krak[i_ob])[['bid_size','bid','ask','ask_size']]
           if ob_krak[i_ob] is not None else None for i_ob in list(ob_krak.keys())}

KeyError: 'bitfinex'

# Alineacion de timestamps

In [5]:
list(ob_krak.keys())[0], list(ob_bit.keys())[0], list(ob_bit.keys())[1], list(ob_bit.keys())[2], list(ob_bit.keys())[3]

('2021-07-05T13:06:49.495Z',
 '2021-07-05T13:06:46.571Z',
 '2021-07-05T13:06:47.918Z',
 '2021-07-05T13:06:49.414Z',
 '2021-07-05T13:06:51.077Z')

In [6]:
# filtro  para que destination sea despues de origin
aux = sum(pd.to_datetime(pd.Series((ob_bit.keys()))) < pd.to_datetime(pd.Series(ob_krak.keys()))[0])
[ob_bit.pop(key) for key in list(ob_bit.keys())[:aux]];

# Origin exchange (Kraken)

In [7]:
# OB 1
df_krak = ob_krak[list(ob_krak.keys())[0]]
# midprice
mid_krak = (df_krak['bid'][0]+df_krak['ask'][0])/2
display(df_krak.head(3))
mid_krak

,bid_size,bid,ask,ask_size
0,0.880,28298.3,28298.4,3.352
1,0.350,28293.9,28300.0,0.020
2,0.377,28293.5,28305.4,0.074


28298.35

In [13]:
upper_krak = mid_krak*1.001
lower_krak = mid_krak*0.999
(lower_krak, upper_krak)

(28270.051649999998, 28326.648349999996)

In [14]:
bids_krak = df_krak[df_krak['bid']>lower_krak][['bid','bid_size']]
asks_krak = df_krak[df_krak['ask']<upper_krak][['ask','ask_size']]

In [15]:
[d.shape for d in [df_krak, bids_krak, asks_krak]]

[(100, 4), (30, 2), (23, 2)]

In [16]:
levels = pd.concat([bids_krak, asks_krak], axis=1)

In [73]:
levels

,bid,bid_size,ask,ask_size
0,28298.3,0.880,28298.4,3.352
1,28293.9,0.350,28300.0,0.020
2,28293.5,0.377,28305.4,0.074
3,28292.2,0.011,28305.5,0.123
4,28292.0,1.221,28307.3,0.204
5,28291.2,0.745,28308.2,0.005
6,28291.0,0.396,28310.0,0.004
7,28290.9,0.340,28311.8,0.745
8,28290.8,0.006,28314.9,0.250
9,28290.0,0.003,28315.2,5.000


# Destination exchange (Bitfinex)

In [20]:
# OB 1
df_bit = ob_bit[list(ob_bit.keys())[0]]
# midprice
mid_bit = (df_bit['bid'][0]+df_bit['ask'][0])/2
display(df_bit.head(3))
mid_bit

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


28276.5

# Añadir niveles a destination

### Bids

In [162]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [163]:
# merge levels to create new OB
bids = df_bit[['bid_size','bid']].merge(levels[['bid_size','bid']], how='outer', on='bid'
                                       ).sort_values(by='bid', ascending=False, ignore_index=True)
bids.rename(columns={'bid_size_x':'bid_size','bid_size_y':'add_size'}, inplace=True)

In [164]:
# identify each order's fee structure
bids['fee_type'] = ['taker' if bid > df_bit['ask'][0] else 'maker' if pd.notna(add_size) else np.nan
                    for bid, add_size in zip(bids['bid'].values, bids['add_size'].values)]

bids.fillna(0, inplace=True)

In [166]:
# compute total volume in each OB level
bids['new_size'] = bids['bid_size'].values+bids['add_size'].values

In [171]:
df_bit[['ask','ask_size']].iloc[0,:]

ask         28278.000000
ask_size        0.011063
Name: 0, dtype: float64

In [172]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [167]:
bids

,bid_size,bid,add_size,fee_type,new_size
0,0.000000,28298.3,0.880,taker,0.880000
1,0.000000,28293.9,0.350,taker,0.350000
2,0.000000,28293.5,0.377,taker,0.377000
3,0.000000,28292.2,0.011,taker,0.011000
4,0.000000,28292.0,1.221,taker,1.221000
5,0.000000,28291.2,0.745,taker,0.745000
6,0.000000,28291.0,0.396,taker,0.396000
7,0.000000,28290.9,0.340,taker,0.340000
8,0.000000,28290.8,0.006,taker,0.006000
9,0.000000,28290.0,0.003,taker,0.003000


In [92]:
df_bit.shape[0],levels.shape[0], test_bid.shape[0]

(25, 30, 55)

### Asks

In [68]:
df_bit.head(3)

,bid_size,bid,ask,ask_size
0,14.605659,28275.0,28278.0,0.011063
1,0.005124,28273.0,28279.0,0.006000
2,0.010158,28272.0,28280.0,0.100000


In [154]:
# merge levels to create new OB
asks = df_bit[['ask','ask_size']].merge(levels[['ask','ask_size']], how='outer', on='ask'
                                       ).sort_values(by='ask', ascending=True, ignore_index=True)

asks.rename(columns={'ask_size_x':'ask_size','ask_size_y':'add_size'}, inplace=True)

In [155]:
# identify each order's fee structure
asks['fee_type'] = ['taker' if ask < df_bit['bid'][0] else 'maker' if pd.notna(add_size) else np.nan
                    for ask, add_size in zip(asks['ask'].values, asks['add_size'].values)]

asks.fillna(0, inplace=True)

In [156]:
# compute total volume in each OB level
asks['new_size'] = asks['ask_size'].values+asks['add_size'].values

In [157]:
asks = asks[asks['new_size']!=0]

In [160]:
asks

,ask,ask_size,add_size,fee_type,new_size
0,28278.0,0.011063,0.000,0,0.011063
1,28279.0,0.006000,0.000,0,0.006000
2,28280.0,0.100000,0.000,0,0.100000
3,28281.0,0.050000,0.000,0,0.050000
4,28282.0,0.323781,0.000,0,0.323781
5,28283.0,1.580312,0.000,0,1.580312
6,28284.0,0.353892,0.000,0,0.353892
7,28285.0,1.053899,0.000,0,1.053899
8,28286.0,0.950994,0.000,0,0.950994
9,28287.0,0.157200,0.000,0,0.157200


In [125]:
#asks['fee'] = [.0015*ask*vol if fee_type=='maker' else .0030*ask*vol if fee_type=='taker' else 0 
#            for ask,vol,fee_type in zip(asks['ask'], asks['add_size'] ,asks['fee_type'])]

<ipython-input-125-85243c79d14a>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  asks['fee'] = [.0015*ask*vol if fee_type=='maker' else .0030*ask*vol if fee_type=='taker' else 0


In [133]:
asks

,ask_size,ask,add_size,fee_type,new_size
0,0.011063,28278.0,0.000,0,0.011063
1,0.006000,28279.0,0.000,0,0.006000
2,0.100000,28280.0,0.000,0,0.100000
3,0.050000,28281.0,0.000,0,0.050000
4,0.323781,28282.0,0.000,0,0.323781
5,1.580312,28283.0,0.000,0,1.580312
6,0.353892,28284.0,0.000,0,0.353892
7,1.053899,28285.0,0.000,0,1.053899
8,0.950994,28286.0,0.000,0,0.950994
9,0.157200,28287.0,0.000,0,0.157200


In [91]:
df_bit.shape[0],levels.shape[0], test_ask.shape[0]

(25, 30, 54)

___

In [60]:
new_df = df_bit.append(levels).reset_index(drop=True)

In [169]:
# create new orderbook
new_df = pd.concat([new_df[['bid_size', 'bid']].sort_values('bid', ascending=False).reset_index(drop=True), 
                   new_df[['ask', 'ask_size']].sort_values('ask', ascending=True).reset_index(drop=True)],
                   axis=1)

# Next steps

1. Insertar liquidez en OB:

    * Proceso de matching engine con ask subsecuente (órdenes límite que crucen el spread hasta que sean agotadas) (taker fee)
    * Calcular comisiones cada vez que se empaten órdenes
    * Calcular efectos de nuestras órdenes y crear un nuevo Orderbook
    
 
2. Comparar OB creado vs OB en siguiente timestamp  

    * Obtener diferencias en niveles para validar si se llenaron nuestras órdenes con el siguiente OB (maker fee)


3. Incorporar proceso de hedge en origin exchange

    * Cada vez que se ejecute una orden en destination, se hae una operación contraria a spot en el origin
    
    
4. Reflejar efectos de hedge en el origin exchange

    * Preguntar al profe